In [1]:
import numpy as np
import pandas as pd

import numpy.linalg as nla

from sklearn.linear_model import LinearRegression

from warnings import warn

# Exercise 17.7

Write a program to implement the modified regression procedure in
Algorithm 17.1 (see main text) for fitting the Gaussian graphical model with pre-specified
edges missing. Test it on the flow cytometry data from the book website,
using the graph of Figure 17.1.

# Solution

Let's introduce the algorithm,

In [2]:
class UndirectedGaussianGraphicalModel():
    
    def __init__(self,vertices,edges,scale=1):
        
        # Graph structure
        self.vertices = vertices
        self.edges = edges
        
        # Scale for the sampled covariance matrix
        self.scale = scale
    
    # For a given vertex, select the indices of those that are connected to it
    def relevant_vertices(self,vertex):
        
        relevant = np.empty(0,dtype=int)
        for u in self.edges[vertex]:
            relevant = np.concatenate((relevant,np.where(self.vertices == u)[0]))

        return relevant
    
    # Get the predictors matrix by removing vertices that are not connected to vertex
    def partition_cov(self,relevant_indices):
        
        W_reduced = self.W[relevant_indices]
        W_reduced = W_reduced[:,relevant_indices]
        
        return W_reduced
    
    # Perform linear regression to invert the matrix
    def get_regression_coefficient(self,j,relevant_indices):

        if relevant_indices.size == 0:
            return np.zeros(self.p-1)
        
        # Get the reduced covariance matrix
        W_reduced = self.partition_cov(relevant_indices)

        # Get the response vector
        S_response = self.S[relevant_indices,j]
        
        # Get the coefficient vector
        model = LinearRegression(fit_intercept=False)
        model.fit(W_reduced,S_response)
        coefficient_reduced = model.coef_
        
        # Get the full coefficient by padding with zeros
        coefficient = np.zeros(self.p)
        coefficient[relevant_indices] = coefficient_reduced
        coefficient = np.delete(coefficient,j)

        return coefficient
    
    # Update the covariance matrix for the j-th vertex
    def update_cov(self,j,coefficient):
        
        # Remove row and column of the relevant vertex
        W_other = np.delete(self.W,j,axis=0)
        W_other = np.delete(W_other,j,axis=1)
        
        # Get updated column/row for the W matrix
        W_update = W_other @ coefficient
        
        # select rows/columns to be updated
        index_update = np.arange(0,self.p,dtype=int) != j

        self.W[j,index_update] = W_update
        self.W[index_update,j] = W_update

    # Compute the inverse of the covariance matrix, (has info on conditional independence)
    def compute_omega(self,coefficients):
        
        self.omega = np.zeros((self.p,self.p))
        
        for j in range(self.p):
            
            # select rows/columns to be updated
            index_update = np.arange(0,self.p,dtype=int) != j
            
            # get the relevant coefficients
            coeff = coefficients[j]
            
            # Update the omega matrix
            self.omega[j,j] = 1/(self.S[j,j] - self.W[j,index_update] @ coeff)
            self.omega[j,index_update] = -self.omega[j,j] * coeff
            self.omega[index_update,j] = -self.omega[j,j] * coeff
    
    # Get the covariance matrix of the graph
    def fit(self,X,is_covariance=False,max_iterations=100):
        
        # compute the sampled covariance matrix
        if is_covariance:
            self.S = X
        else:
            self.S = np.cov(X.T)/self.scale
            
        # Initialize W matrix (constrained covariance matrix)
        self.W = self.S.copy()
        
        _, self.p = X.shape
        
        not_converged = True
        coefficients = {} # Store coefficient list for last step of algorithm
        
        for _ in range(max_iterations):
            
            previous_W = self.W.copy()
        
            for j,vertex in enumerate(self.vertices):

                # Get the relevant indices for this vertex
                relevant_indices = self.relevant_vertices(vertex)

                # Get the regression coefficient
                coefficients[j] = self.get_regression_coefficient(j,relevant_indices)

                # Update the covariance matrix
                self.update_cov(j,coefficients[j])
               
            if np.allclose(previous_W,self.W):
                not_converged = False
                break
                
        if not_converged:
            warn("Method has not converged. Try increasing the max_iteration parameter.")
        
        # Compute inverse of covariance matrix
        self.compute_omega(coefficients)

### Test case

We can test the algorithm on the covariance matrix given in the main text (Figure 17.4),

In [3]:
# Test covariance matrix
S = np.array([[10,1,5,4],[1,10,2,6],[5,2,10,3],[4,6,3,10]],dtype=float)

# Test graph
vertices = np.array([1,2,3,4])

edges = {}
edges[1] = [2,4]
edges[2] = [1,3]
edges[3] = [2,4]
edges[4] = [1,3]

# Let's fit the model and obtain the constrained covariance matrix (and it's inverse)
UGGM = UndirectedGaussianGraphicalModel(vertices,edges)
UGGM.fit(S,is_covariance=True)

In [4]:
# Constrained covariance matrix
print(UGGM.W.round(2))

[[10.    1.    1.31  4.  ]
 [ 1.   10.    2.    0.87]
 [ 1.31  2.   10.    3.  ]
 [ 4.    0.87  3.   10.  ]]


In [5]:
# Constrained inverse covariance matrix
print(UGGM.omega.round(2))

[[ 0.12 -0.01 -0.   -0.05]
 [-0.01  0.1  -0.02 -0.  ]
 [-0.   -0.02  0.11 -0.03]
 [-0.05 -0.   -0.03  0.13]]


The test is succesfull.

## Main dataset (protein)

The undirected graph is given, see figure 17.1, and we report here the name of the nodes and the edges,

In [6]:
vertices = np.array(["praf","pmek","plcg","PIP2","PIP3","p44/42","pakts473","PKA","PKC","P38","pjnk"])

edges = {}

edges["praf"] = ["pmek"]
edges["pmek"] = ["praf","P38","PKA","pakts473","PIP2"]
edges["plcg"] = ["P38","pakts473","PIP2"]
edges["PIP2"] = ["plcg","pjnk","P38","PKA","pakts473"]
edges["PIP3"] = []
edges["p44/42"] = []
edges["pakts473"] = ["PIP2","pmek","P38"]
edges["PKA"] = ["PIP2","plcg","pmek","P38"]
edges["PKC"] = ["P38"]
edges["P38"] = ["pjnk","pmek","plcg","PIP2","pakts473","PKA","PKC"]
edges["pjnk"] = ["PIP2","P38"]


Let's load the data,

In [7]:
url_link = 'https://web.stanford.edu/~hastie/ElemStatLearn/datasets/sachs.data'
df = pd.read_csv(url_link,sep=' ',header=None,names=vertices)

df.tail()

,praf,pmek,plcg,PIP2,PIP3,p44/42,pakts473,PKA,PKC,P38,pjnk
7461,-74.97193,-132.981,-22.05364,-123.22070,-4.334962,-14.931190,-42.96721,518.2414,-29.34166,-132.4645,-72.2675
7462,-100.77190,-140.771,-37.05364,-129.02070,-12.134960,22.068810,-13.86721,296.2414,-29.34166,-125.1945,-72.2675
7463,-95.97193,-140.891,-36.05364,-130.92070,-16.834960,-23.551190,-59.26721,104.2414,-29.34166,-133.2645,-71.2675
7464,-89.47193,-138.281,-49.12364,-130.42070,-11.934960,5.568807,-39.76721,187.2414,14.15834,1246.9850,-70.8275
7465,-93.57193,-144.371,-47.55364,21.87926,-4.134962,-20.021190,-67.46721,264.2414,-29.34166,-134.0145,-71.6175


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7466 entries, 0 to 7465
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   praf      7466 non-null   float64
 1   pmek      7466 non-null   float64
 2   plcg      7466 non-null   float64
 3   PIP2      7466 non-null   float64
 4   PIP3      7466 non-null   float64
 5   p44/42    7466 non-null   float64
 6   pakts473  7466 non-null   float64
 7   PKA       7466 non-null   float64
 8   PKC       7466 non-null   float64
 9   P38       7466 non-null   float64
 10  pjnk      7466 non-null   float64
dtypes: float64(11)
memory usage: 641.7 KB


In [9]:
X = df.to_numpy()

Let's use the above algorithm to estimate the covariance matrix of the graph in Figure 17.1, and in particual visualize the inverse covariance matrix that carries info on the conditional independence of the vertices,

In [16]:
UGGM = UndirectedGaussianGraphicalModel(vertices,edges,scale=1000)
UGGM.fit(X)

print(UGGM.omega)

[[ 8.40086124e-01 -5.46112280e-01 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-5.46112280e-01  3.63557496e-01 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -4.86499268e-03  4.49344942e-04
  -0.00000000e+00 -6.11490345e-04 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00  2.37660373e-01 -1.25117134e-01
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.38353925e-03
  -0.00000000e+00 -3.97985846e-03 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.25117134e-01  7.91464408e-02
  -0.00000000e+00 -0.00000000e+00 -9.17018779e-03 -1.08855030e-04
  -0.00000000e+00  1.92574929e-03 -4.94588823e-03]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
   5.39623518e-01 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00  